In [1]:
import pandas as pd
dat= pd.read_csv('head_base_mapping.csv')
dat.shape[0]

414

In [10]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import PIL
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
%matplotlib inline
from annoy import AnnoyIndex


imsize=224
### resnet stuff
extractor_resnet = models.resnet101(pretrained=True)
extractor_resnet.eval()
num_ftrs = 1000

loader = transforms.Compose([transforms.Resize((imsize,imsize)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    #image_loader_1 = PIL.Image.fromarray(image_name)
    image_loader_1 = PIL.Image.open(image_name).convert("RGB")
    image_loader_1 = loader(image_loader_1).float()
    image_loader_1 = Variable(image_loader_1, requires_grad=True)
    image_loader_1 = image_loader_1.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    y_pred = extractor_resnet(image_loader_1)
    return y_pred.tolist()[0]#.cuda()  #assumes that you're using GPU


def showImagesHorizontally(annoy_dict, base_img, dir_, list_of_files,index):
    fig = figure(figsize=(160,90))
    number_of_files = len(list_of_files)
    list_of_files = [base_img] + list_of_files
    first_bool = 0 
    for i in range(number_of_files):
        if first_bool ==0:
            a=fig.add_subplot(1,number_of_files,i+1)
            image = imread(list_of_files[i])
            first_bool = 1
            
        else:
            a=fig.add_subplot(1,number_of_files,i+1)
            image = imread(dir_+annoy_dict[str(list_of_files[i])])
        
        #image = imread(dir_l+annoy_dict[str(list_of_files[i])])
        imshow(image,aspect='equal')
        axis('off')
    fig.savefig('F:/fgo_multi_label/sim_examples/{}.png'.format(index))
    fig.clf()
    

In [3]:
from os import listdir
img_dir ='D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/all_images/'
img_list = list(listdir(img_dir))
#print(feature_vec.view(-1).tolist())

In [4]:
print('euclidean')
annoy_index = AnnoyIndex(num_ftrs,metric='euclidean')

annoy_list = []
index = 0

for img in img_list:
    img_name = img
    try:
        feature_vec = image_loader(img_dir+img_name)
        annoy_index.add_item(index, feature_vec)
        annoy_list.append(index)
        index+=1
    except:
        print('error on image {}'.format(img_name))
    


annoy_index.build(200)
    #dat.head()
    #dat.to_csv('mapping_with_annoy.csv')


euclidean


True

In [11]:
dict_fgo_annoy = {}
dat= pd.DataFrame({'original_image':img_list,'annoy_index':annoy_list})
for index,row in dat.iterrows():
    dict_fgo_annoy[str(row['annoy_index'])] = str(row['original_image']) #switch this out

dir_ = 'F:/fgo_multi_label/test/'

target_dir = 'F:/fgo_multi_label/test/'
file_list = []
for i in listdir(target_dir):
    if 'xml' not in i[-4:]:
        file_list.append(i)
        
#test_imgs = ['__artoria_pendragon_and_saber_fate_grand_order_and_etc_drawn_by_mirea__0195ad0980d81a8eb26097cb5e42ad7b.jpg',
#            '0d5c845e15a82b6cd90aaeeb5efd28771749b642_00.jpg','1caefaeaf73902bf7cdd985bcda14001.jpg',
#            '06e356adef566015ae1ca6283b8d668b--anime-warrior-fate-grand-order.jpg','8d5fde991026b80c48b72ae3bfa3cf7b.jpg']
index = 0 
for i in file_list:
    img_name = i
    feature_vec = image_loader(dir_+img_name)
    test_list = annoy_index.get_nns_by_vector(feature_vec, 5)
    base_im_path = dir_+img_name
    showImagesHorizontally(annoy_dict=dict_fgo_annoy,base_img = base_im_path,dir_=img_dir,list_of_files = test_list,index=index)
    index+=1

d:\users\micha\appdata\local\programs\python\python35\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
